In [6]:
from torch_geometric.datasets import Planetoid
import os.path as osp
import torch_geometric.transforms as T
import torch
from torch_geometric import utils
from torch_geometric.data import Data

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.eval_metrics import *

dataset = 'Cora'
path = osp.join('..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

# data.edge_index, data.edge_weight = gcn_norm(  
#                 data.edge_index, data.edge_weight, data.num_nodes,
#                 add_self_loops=False, dtype=data.x.dtype)
from sklearn.decomposition import PCA

def reduce_dimensions(features, new_dim=64):
    pca = PCA(n_components=new_dim)
    reduced_features = pca.fit_transform(features)
    return torch.tensor(reduced_features, dtype=torch.float)

delta = 0.85
edge_index, edge_weight = utils.get_laplacian(data.edge_index, data.edge_weight, normalization='sym')
L = utils.to_dense_adj(edge_index, edge_attr=edge_weight)
A = torch.eye(data.num_nodes) - delta*L
data.edge_index, data.edge_weight = utils.dense_to_sparse(A)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
data = data.to(device)
original_data = Data(x=reduce_dimensions(data.x.numpy()), edge_index=data.edge_index)

In [7]:
import torch
edge_index = dataset[0].edge_index
num_nodes = dataset[0].num_nodes
adjacency_matrix = torch.zeros((num_nodes, num_nodes), dtype=torch.float)
adjacency_matrix[edge_index[0], edge_index[1]] = 1


In [8]:
from sklearn.cluster import SpectralClustering
adjacency_matrix_np = adjacency_matrix.numpy()
clustering = SpectralClustering(n_clusters=dataset.num_classes, affinity='precomputed', assign_labels='discretize').fit(adjacency_matrix_np)
labels = clustering.labels_


/opt/homebrew/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [11]:
from sklearn.metrics import normalized_mutual_info_score
labels_true = dataset[0].y
acc, nmi = eval_metrics(torch.tensor(labels_true), torch.tensor(labels))
print(f'NMI: {nmi:.3f}, ACC: {acc:.3f}')

NMI: 0.149, ACC: 0.335


/var/folders/st/cyh3v8ln22sdswp1s386w5vw0000gn/T/ipykernel_51405/3975282371.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  acc, nmi = eval_metrics(torch.tensor(labels_true), torch.tensor(labels))
